In [1]:
###################################################
##              Import libraries                 ##
###################################################

import pandas as pd
import numpy as np
import itertools
from collections import defaultdict
import  math

In [2]:
######################################################################
##                        Read the files                            ##
######################################################################
data = {}
for file in ['chania','rethymno','heraklion','lasithi']:
    data[file] = pd.read_excel(f'/Users/vasou/Documents/HLA/DHTOB/{file}.xlsx')
    
#all_cretans = pd.read_excel(f'/Users/vasou/Documents/HLA/1744_COMPLETE_CRETANS.xlsx')
#all_cretans.rename(columns = {'#':'id'}, inplace = True)
#all_cretans.replace('?',np.NaN, inplace = True)

In [3]:
##############################################################################  
##                 Found the alleles for all regions                        ##
##############################################################################

def unique_and_sorted_allele(data, #Dictionary: key (string) = region (chania,rethymno etc), value(dataframe) = allleles
                            ):
    
    alleles_all_regions = defaultdict(list)
    
    for region in data:
        for allele in ['A', 'B', 'C', 'DRB1','DQB1','DPB1']:#set([i.split('_')[0] if '_' in i else i[0] for i in data[key].columns[1:]]):
            alleles_all_regions[allele].append(list(pd.unique(
                                                data[region][[allele+'_1' if 'D' in allele else allele + '1',
                                                allele+'_2' if 'D' in allele else allele + '2']].values.ravel())))

    alleles_all_regions = {allele:sorted(set(list(itertools.chain(*alleles_all_regions[allele])))) for allele in alleles_all_regions}
    
    return alleles_all_regions

In [4]:
#############################################################################################  
##                            Allele and Genotype counts                                   ##
#############################################################################################


def allele_genotype_counts(data, #Dictionary: key = region(chania,rethymno etc), value(dataframe) = allleles
                           type_count = 'Allele' #Allele: returns allele counts, Genotype:returns observed genotype counts
                          ):
    
    count_dict = defaultdict(lambda:defaultdict())
    
    ########################################################
    ##   First found the unique alleles from all regions  ##
    ########################################################
    alleles_all_regions = unique_and_sorted_allele(data)

    for region in data:
        for allele in alleles_all_regions:
            column_1 = allele + '_1' if 'D' in allele else allele + '1'
            column_2 = allele + '_2' if 'D' in allele else allele + '2'
            total_het = 0
            total_hom = 0
            for i in range(len(alleles_all_regions[allele])):

                #Check Hom
                count_hom = sum((data[region][column_1] == alleles_all_regions[allele][i]) & (
                            data[region][column_2] == alleles_all_regions[allele][i]))
                
                if type_count == 'Allele': 
                    count_hom = 2*count_hom
                    
                    #Het
                    count_het_1 = sum((data[region][column_1] == alleles_all_regions[allele][i]))
                    count_het_2 = sum((data[region][column_2] == alleles_all_regions[allele][i]))
                    count_het = (count_het_1 + count_het_2) - count_hom
                else:
                    #Het
                    count_het = sum((data[region][column_1] == alleles_all_regions[allele][i])) - count_hom
                    
                total_het += count_het
                total_hom += count_hom
                
                count_dict[f'hom_{region}_{allele}'][alleles_all_regions[allele][i]] = count_hom
                
                #Store Het
                count_dict[f'het_{region}_{allele}'][alleles_all_regions[allele][i]] = count_het
                
            count_dict[f'n_hom_{allele}_{region}'] = total_hom
            count_dict[f'n_het_{allele}_{region}'] = total_het
                        

            #for i in range(len(alleles_all_regions[allele])):
            #    
            #    count_dict[f'het_rest_{region}_{allele}'][alleles_all_regions[allele][i]] = total_het - count_dict[f'het_{region}_{allele}'][alleles_all_regions[allele][i]]
            #    count_dict[f'hom_rest_{region}_{allele}'][alleles_all_regions[allele][i]] = total_hom - count_dict[f'hom_{region}_{allele}'][alleles_all_regions[allele][i]]
    
    
    for allele in alleles_all_regions:
        hom_total = 0
        het_total = 0
        for i in range(len(alleles_all_regions[allele])):
            hom = 0
            het = 0
            for region in data:
                hom += count_dict[f'hom_{region}_{allele}'][alleles_all_regions[allele][i]]
                het += count_dict[f'het_{region}_{allele}'][alleles_all_regions[allele][i]]
                
                 
                count_dict[f'het_rest_{region}_{allele}'][alleles_all_regions[allele][i]] = count_dict[f'n_het_{allele}_{region}'] - count_dict[f'het_{region}_{allele}'][alleles_all_regions[allele][i]]
                count_dict[f'hom_rest_{region}_{allele}'][alleles_all_regions[allele][i]] = count_dict[f'n_hom_{allele}_{region}'] - count_dict[f'hom_{region}_{allele}'][alleles_all_regions[allele][i]]

                    
            count_dict[f'hom_total_{allele}'][alleles_all_regions[allele][i]] = hom 
            count_dict[f'het_total_{allele}'][alleles_all_regions[allele][i]] = het
            hom_total += hom
            het_total += het
            
        count_dict[f'n_hom_total_{allele}'] = hom_total
        count_dict[f'n_het_total_{allele}'] = het_total
            

        for i in range(len(alleles_all_regions[allele])):

            count_dict[f'hom_rest_total_{allele}'][alleles_all_regions[allele][i]] = hom_total - count_dict[f'hom_total_{allele}'][alleles_all_regions[allele][i]]
            count_dict[f'het_rest_total_{allele}'][alleles_all_regions[allele][i]] = het_total - count_dict[f'het_total_{allele}'][alleles_all_regions[allele][i]]
           
    return count_dict


#allele_count = allele_genotype_counts(data, type_count = 'Allele')
#obs_gen_count = allele_genotype_counts(data, type_count = 'Genotype')



In [5]:
#######################################################################################
##                         Step 1: Allele Frequencies                                ##
#######################################################################################


def help_func_freq(a,b,c,d, allele_freq, key_part1, key_part2):
    nominator = a +b
    denominator = c + d
    calc_p = nominator/denominator
    
    allele_freq[f'p_{key_part1}'][key_part2] = calc_p
    allele_freq[f'p^2_{key_part1}'][key_part2] = calc_p * calc_p
    allele_freq[f'q_{key_part1}'][key_part2] = 1 - calc_p
    allele_freq[f'q^2_{key_part1}'][key_part2] = (1 - calc_p)**2
    
    
def allele_frequencies(data #Dictionary: key (string) = region (chania,rethymno etc), value(dataframe) = allleles
                      ):
    
    n_allele = defaultdict(lambda: defaultdict(lambda: 0))
    allele_freq = defaultdict(lambda: defaultdict(lambda: 0))
    
    
    ########################################################
    ##   First found the unique alleles from all regions  ##
    ########################################################
    alleles_all_regions = unique_and_sorted_allele(data)
    
    ########################################################
    ##                   Allele counts                    ##
    ########################################################
    allele_count = allele_genotype_counts(data, type_count = 'Allele')

    ########################################################
    ##              Calculate frequencies                 ##
    ########################################################

    for allele in alleles_all_regions:
        for i in range(len(alleles_all_regions[allele])):
            total_samples = 0
            for region in data:
                total_samples += data[region].shape[0]
                help_func_freq(allele_count[f'hom_{region}_{allele}'][alleles_all_regions[allele][i]],
                          allele_count[f'het_{region}_{allele}'][alleles_all_regions[allele][i]],
                          allele_count[f'n_hom_{allele}_{region}'],
                          allele_count[f'n_het_{allele}_{region}'], 
                          allele_freq, 
                          f"{region}_{allele}", 
                          alleles_all_regions[allele][i])
                
               
            help_func_freq(allele_count[f'hom_total_{allele}'][alleles_all_regions[allele][i]],
                      allele_count[f'het_total_{allele}'][alleles_all_regions[allele][i]],
                      allele_count[f'n_hom_total_{allele}'], 
                      allele_count[f'n_het_total_{allele}'],
                      allele_freq,
                      f"total_{allele}", 
                      alleles_all_regions[allele][i])


    return allele_freq


#allele_freq = allele_frequencies(data)


In [6]:
#######################################################################################
##             Step 2: expected genotypic counts (under HWE)                         ##
#######################################################################################


def help_func_hwe(a,b,c,d,counts_HWE, key_part1, key_part2):
    counts_HWE[f'hom_p^2_{key_part1}'][key_part2] = a*b
    counts_HWE[f'het_2pq_{key_part1}'][key_part2] = 2*c*d*b

def HWE(data):
    counts_HWE = defaultdict(lambda: defaultdict(lambda: 0))
    
    ########################################################
    ##   First found the unique alleles from all regions  ##
    ########################################################
    alleles_all_regions = unique_and_sorted_allele(data)
    
    ########################################################
    ##            Calculate allele frequencies            ##
    ########################################################
    allele_freq = allele_frequencies(data)
    
    for allele in alleles_all_regions:
        for i in range(len(alleles_all_regions[allele])):
            total_samples = 0
            for region in data:
                total_samples += data[region].shape[0]
                
                help_func_hwe(allele_freq[f'p^2_{region}_{allele}'][alleles_all_regions[allele][i]],
                              data[region].shape[0],
                              allele_freq[f'p_{region}_{allele}'][alleles_all_regions[allele][i]],
                              allele_freq[f'q_{region}_{allele}'][alleles_all_regions[allele][i]],
                              counts_HWE,
                              f'{region}_{allele}',
                              alleles_all_regions[allele][i] 
                             )
                
            help_func_hwe(allele_freq[f'p^2_total_{allele}'][alleles_all_regions[allele][i]],
                          total_samples,
                          allele_freq[f'p_total_{allele}'][alleles_all_regions[allele][i]],
                          allele_freq[f'q_total_{allele}'][alleles_all_regions[allele][i]],
                          counts_HWE,
                          f'total_{allele}',
                          alleles_all_regions[allele][i] 
                         )
    return counts_HWE
              
#counts_HWE = HWE(data)

In [7]:
#######################################################################################
##                 Step 3: excess or deficiencies of observed                        ##
##      homozygotes in each subpopulation relative to HWE genotypic counts           ##
#######################################################################################

def deficiencies_sub(data):
    obs_exp = defaultdict(lambda: defaultdict(lambda: 0))
    
    ########################################################
    ##   First found the unique alleles from all regions  ##
    ########################################################
    alleles_all_regions = unique_and_sorted_allele(data)
    
    ########################################################
    ##                   Allele counts                    ##
    ########################################################
    obs_gen_count = allele_genotype_counts(data, type_count = 'Genotype')
    
    ########################################################
    ##                   Calculate HWE                    ##
    ########################################################
    counts_HWE = HWE(data)
    
    ########################################################
    ##             Calculate deficiencies                 ##
    ########################################################
    for allele in alleles_all_regions:
        for i in range(len(alleles_all_regions[allele])):
            total_samples = 0
            for region in data:
    
                obs_exp[f'obs-exp_p^2_{region}_{allele}'][alleles_all_regions[allele][i]] = (
                                obs_gen_count[f'hom_{region}_{allele}'][alleles_all_regions[allele][i]] -
                                counts_HWE[f'hom_p^2_{region}_{allele}'][alleles_all_regions[allele][i]])
            obs_exp[f'obs-exp_p^2_total_{allele}'][alleles_all_regions[allele][i]] = (
                            obs_gen_count[f'hom_total_{allele}'][alleles_all_regions[allele][i]]-
                            counts_HWE[f'hom_p^2_total_{allele}'][alleles_all_regions[allele][i]])

    return obs_exp


#excess = deficiencies_sub(data)


In [8]:
#######################################################################################
##   Step4: excess or deficiencies of observed homozygotes in the total population   ##
#######################################################################################

def deficiencies_total(data):
    sum_obs_exp = defaultdict(lambda: 0)

    ########################################################
    ##          Calculate deficiencies_sub                ##
    ########################################################
    excess = deficiencies_sub(data)
    
    ########################################################
    ##             Calculate deficiencies                 ##
    ########################################################
    for key in excess:

        sum_obs_exp[f'sum_{key}'] = round(sum(excess[key].values()))

    return sum_obs_exp


In [9]:
#######################################################################################
##             Step 5: local observed heterozygosities of each subpopulation         ##
#######################################################################################


def obs_heterozygosities(data):
    obs_het = defaultdict(lambda:0)
    
    ########################################################
    ##                   Allele counts                    ##
    ########################################################
    obs_gen_count = allele_genotype_counts(data, type_count = 'Genotype')
    
    ########################################################
    ##           local observed heterozygosities          ##
    ########################################################
    
    for allele in data['chania'].columns[1:]:
        total_samples = 0
        for region in data:
            total_samples+=data[region].shape[0]
            allele = allele.split('_')[0] if 'D' in allele else allele[0]
            obs_het[f'Het_{region}_{allele}'] = obs_gen_count[f'n_het_{allele}_{region}']/data[region].shape[0]
        obs_het[f'Het_total_{allele}'] = obs_gen_count[f'n_het_total_{allele}']/total_samples
            
    return obs_het


#obs_het = obs_heterozygosities(data)


In [10]:
#######################################################################################
##               Step 6: expected heterozygosities of each subpopulation             ##
#######################################################################################

def exp_heterozygosities(data):
    exp_het = defaultdict(lambda:0)
    ########################################################
    ##            Calculate allele frequencies            ##
    ########################################################
    allele_freq = allele_frequencies(data)
    
    for allele in data['chania'].columns[1:]:
        allele = allele.split('_')[0] if 'D' in allele else allele[0]
        for region in data:
            exp_het[f'Het_{region}_{allele}'] = 1 - sum(allele_freq[f'p^2_{region}_{allele}'].values())
        exp_het[f'Het_total_{allele}'] = 1 - sum(allele_freq[f'p^2_total_{allele}'].values())
    return exp_het


#expected_het = exp_heterozygosities(data)

In [11]:
#######################################################################################
##  Step 7: local inbreeding coefficient of each subpopulation (Fs=Hexp-Hobs/Hexp)   ##
#######################################################################################

def Fs_coefficient(data):
    
    Fs = defaultdict(lambda: 0)
    
    ########################################################
    ##         Calculate expected heterozygosities        ##
    ########################################################
    expected_het = exp_heterozygosities(data)
    
    ########################################################
    ##         Calculate observed heterozygosities        ##
    ########################################################
    obs_het = obs_heterozygosities(data)
    
    for allele in data['chania'].columns[1:]:
        allele = allele.split('_')[0] if 'D' in allele else allele[0]
        for region in data:
            Fs[f'Fs_{region}_{allele}'] = (expected_het[f'Het_{region}_{allele}'] - 
                                           obs_het[f'Het_{region}_{allele}'])/(
                                           expected_het[f'Het_{region}_{allele}'])
            
        Fs[f'Fs_total_{allele}'] = (expected_het[f'Het_total_{allele}'] - 
                                           obs_het[f'Het_total_{allele}'])/(
                                            expected_het[f'Het_total_{allele}'])
            
    return Fs
            

#fs_coef = Fs_coefficient(data)

In [42]:
#######################################################################################
##            Step8: p_allele bar (global frequency of each allele                   ##
##       across all subpopulations and weighted by each subpopulation size)          ##
#######################################################################################


def help_func_bar(a,b,c,d,p_allele_bar, key_part1, key_part2):
    
    if d == 0:
        denominator = b
        nominator = c*a
        p_allele_bar[f'{key_part1}'][key_part2] += nominator/denominator
        p_allele_bar[f'{key_part1}_square'][key_part2] = p_allele_bar[f'{key_part1}'][key_part2]**2
    else: 
        denominator = a+b
        nominator = c*a + d*b
        p_allele_bar[f'{key_part1}'][key_part2] = nominator/denominator
        p_allele_bar[f'{key_part1}_square'][key_part2] = (nominator/denominator)**2

def p_allele_bar(data):
    n_allele = defaultdict(lambda: 0)
    p_allele_bar = defaultdict(lambda: defaultdict(lambda: 0))
    
    ########################################################
    ##   First found the unique alleles from all regions  ##
    ########################################################
    alleles_all_regions = unique_and_sorted_allele(data)
    
    ########################################################
    ##            Calculate allele frequencies            ##
    ########################################################
    allele_freq = allele_frequencies(data)
    
    ########################################################
    ##                   Allele counts                    ##
    ########################################################
    allele_count = allele_genotype_counts(data, type_count = 'Allele')
    
    total_samples = sum([data[region].shape[0] for region in data])*2
    for allele in alleles_all_regions:
        for region in data:
            for i in range(len(alleles_all_regions[allele])):
                
                help_func_bar(data[region].shape[0]*2,
                              total_samples,
                              allele_freq[f'p_{region}_{allele}'][alleles_all_regions[allele][i]],
                              0,
                              p_allele_bar,
                              f'all_{allele}',
                              alleles_all_regions[allele][i]
                             )


        for i in range(len(alleles_all_regions[allele])):
            for region_1, region_2 in list(itertools.combinations(data.keys(), 2)):

                help_func_bar(data[region_1].shape[0]*2,
                              data[region_2].shape[0]*2,
                              allele_freq[f'p_{region_1}_{allele}'][alleles_all_regions[allele][i]],
                              allele_freq[f'p_{region_2}_{allele}'][alleles_all_regions[allele][i]],
                              p_allele_bar,
                              f'{region_1}_{region_2}_{allele}',
                              alleles_all_regions[allele][i]
                             )

    return p_allele_bar

In [78]:
#######################################################################################
##                   Step9&10&11: heterozygosity indices                             ##
##              (Hi=weighted average of observed heterozygosities),                  ##
##              (Hs =weighted average of expected heterozygosities),                 ##
## (Ht=expected hetrozygosty based on the global gene frequencies i.e. p_allele-bar) ##
#######################################################################################

def heterozygosity_indices(data):
    
    Het_indices = defaultdict(lambda: 0)
    
    ########################################################
    ##         Calculate expected heterozygosities        ##
    ########################################################
    expected_het = exp_heterozygosities(data)
    
    ########################################################
    ##         Calculate observed heterozygosities        ##
    ########################################################
    obs_het = obs_heterozygosities(data)

    ########################################################
    ##         Calculate observed heterozygosities        ##
    ########################################################
    allele_bar = p_allele_bar(data)
    
    
    for allele in data['chania'].columns[1:]:
        total_samples = 0
        allele = allele.split('_')[0] if 'D' in allele else allele[0]
        for region in data:
            total_samples+=data[region].shape[0]
            Het_indices[f'Hi_all_{allele}'] += (obs_het[f'Het_{region}_{allele}'] * data[region].shape[0])
            Het_indices[f'Hs_all_{allele}'] += (expected_het[f'Het_{region}_{allele}'] * data[region].shape[0])
            
        Het_indices[f'Hi_all_{allele}'] /= total_samples
        Het_indices[f'Hs_all_{allele}'] /= total_samples
        Het_indices[f'Ht_all_{allele}'] = 1 - sum(allele_bar[f'all_{allele}_square'].values())
        

        for region_1, region_2 in list(itertools.combinations(data.keys(), 2)):  
            Het_indices[f'Hi_{region_1}_{region_2}_{allele}'] = (
                                                    (obs_het[f'Het_{region_1}_{allele}'] * data[region_1].shape[0])
                                                     + (obs_het[f'Het_{region_2}_{allele}'] * data[region_2].shape[0]))\
                                                     / (data[region_1].shape[0] + data[region_2].shape[0])

            Het_indices[f'Hs_{region_1}_{region_2}_{allele}'] = (
                                                    (expected_het[f'Het_{region_1}_{allele}'] * data[region_1].shape[0])
                                                 + (expected_het[f'Het_{region_2}_{allele}'] * data[region_2].shape[0]))\
                                                 / (data[region_1].shape[0] + data[region_2].shape[0])
            Het_indices[f'Ht_{region_1}_{region_2}_{allele}'] = 1 - \
                                    sum(allele_bar[f'{region_1}_{region_2}_{allele}_square'].values())
            
    return Het_indices
 
    
    
Het_indices = heterozygosity_indices(data)


In [77]:
#######################################################################################
##           Step12: global Fstatistics (FIS_observed heterozygosities,              ## 
##                    FST_expected heterozygosities,                                 ##
##                    FIT_expected heterozygosities)                                 ##
#######################################################################################

def help_func_Fstats(a,b,c, Fstatistics, key_part):
    
    Fstatistics[f'Fis_{key_part}'] = (a-b)/a
    Fstatistics[f'Fst_{key_part}'] = (c-a)/c
    Fstatistics[f'Fit_{key_part}'] = (c-b)/c

def global_Fstatistics(data):
    
    Fstatistics = defaultdict(lambda:0)
    
    ########################################################
    ##          Calculate heterozygosity indices          ##
    ########################################################
    Het_indices = heterozygosity_indices(data)
    
    for allele in data['chania'].columns[1:]:
        allele = allele.split('_')[0] if 'D' in allele else allele[0]
        help_func_Fstats(Het_indices[f'Hs_all_{allele}'],
                         Het_indices[f'Hi_all_{allele}'],
                         Het_indices[f'Ht_all_{allele}'],
                         Fstatistics,
                         f'all_{allele}',
                         )
        
        for region_1, region_2 in list(itertools.combinations(data.keys(), 2)):
            
            help_func_Fstats(Het_indices[f'Hs_{region_1}_{region_2}_{allele}'],
                             Het_indices[f'Hi_{region_1}_{region_2}_{allele}'],
                             Het_indices[f'Ht_{region_1}_{region_2}_{allele}'],
                             Fstatistics,
                             f'_{region_1}_{region_2}_{allele}',
                            )
    return Fstatistics
    
        
#Fstatistics = global_Fstatistics(data)


In [ ]:
#######################################################################################
##  Step4: excess or deficiencies of observed homozygotes in the total population
#######################################################################################


sum_obs-exp_p^2_Ch = 8 ????????


